# Aplicación en Capas: Índice de Masa Corporal (IMC)

## Introducción

El **Índice de Masa Corporal (IMC)** es una medida que se utiliza para evaluar si una persona tiene un peso saludable en relación con su altura. Se calcula mediante la siguiente fórmula:

$$
IMC = \frac{\text{Peso (kg)}}{\text{Altura (m)}^2}
$$

Donde:
- **Peso**: es el peso de la persona en kilogramos (kg).
- **Altura**: es la altura de la persona en metros (m).

El resultado del IMC nos da una indicación general sobre el estado de salud de una persona. Los valores comunes del IMC se interpretan de la siguiente manera:

- **< 18.5**: Bajo peso.
- **18.5 - 24.9**: Peso normal.
- **25 - 29.9**: Sobrepeso.
- **30 o más**: Obesidad.

## 1 - Capa de Datos: Definición en SQL Server

Para almacenar los datos de las personas y poder calcular su IMC, crearemos una tabla en **SQL Server**. Esta tabla almacenará información básica como el nombre, edad, peso y altura de las personas.

### 1.1 Creación de la Tabla en SQL Server

```sql
CREATE TABLE Personas (
    Id INT PRIMARY KEY IDENTITY(1,1),
    Nombre NVARCHAR(100),
    Edad INT,
    Peso DECIMAL(5,2),  -- Peso en kilogramos
    Altura DECIMAL(4,2) -- Altura en metros
);

--Data de Prueba
INSERT INTO Personas (Nombre, Edad, Peso, Altura)
VALUES 
('Juan Pérez', 28, 72.5, 1.75),
('Ana Gómez', 34, 60.0, 1.68),
('Carlos López', 45, 85.3, 1.80),
('Lucía Martínez', 22, 55.2, 1.62);

-- Consulta
SELECT * FROM Personas;

-- Cáclulo IMC
SELECT 
    Nombre,
    Edad,
    Peso,
    Altura,
    (Peso / (Altura * Altura)) AS IMC
FROM Personas;

-- Cálculo Desviación Estandar
SELECT 
    STDEV(Peso / (Altura * Altura)) AS DesviacionEstandarIMC
FROM Personas;


## 2 Capa Lógica: Cálculo del IMC y Desviación Estándar en C#

En esta capa vamos a implementar la lógica en **C#** para calcular el **Índice de Masa Corporal (IMC)** de cada persona y la **desviación estándar** de los IMC almacenados en nuestra base de datos.

### 2.1 Conexión a la Base de Datos

Primero, necesitamos configurar la conexión con SQL Server en **C#** usando `System.Data.SqlClient`. Asegúrate de instalar las dependencias necesarias en tu proyecto.

#### Código para conectar a la base de datos:

```csharp
using System;
using System.Data.SqlClient;

public class DatabaseConnection
{
    private string connectionString = "Data Source=nombre_del_servidor;Initial Catalog=nombre_de_la_base_de_datos;User ID=usuario;Password=contraseña";

    public SqlConnection Connect()
    {
        SqlConnection connection = new SqlConnection(connectionString);
        try
        {
            connection.Open();
            Console.WriteLine("Conexión a la base de datos exitosa.");
        }
        catch (Exception ex)
        {
            Console.WriteLine("Error al conectar: " + ex.Message);
        }
        return connection;
    }
}


### 2.2 Función para calcular IMC 

```csharp
using System;
using System.Collections.Generic;
using System.Data.SqlClient;

public class Persona
{
    public string Nombre { get; set; }
    public int Edad { get; set; }
    public decimal Peso { get; set; }
    public decimal Altura { get; set; }
    public decimal IMC { get; set; }
}

public class IMCCalculator
{
    public List<Persona> ObtenerPersonasConIMC(SqlConnection connection)
    {
        List<Persona> personas = new List<Persona>();

        string query = "SELECT Nombre, Edad, Peso, Altura FROM Personas";
        SqlCommand command = new SqlCommand(query, connection);
        SqlDataReader reader = command.ExecuteReader();

        while (reader.Read())
        {
            Persona persona = new Persona();
            persona.Nombre = reader["Nombre"].ToString();
            persona.Edad = Convert.ToInt32(reader["Edad"]);
            persona.Peso = Convert.ToDecimal(reader["Peso"]);
            persona.Altura = Convert.ToDecimal(reader["Altura"]);
            
            // Calcular el IMC
            persona.IMC = persona.Peso / (persona.Altura * persona.Altura);
            
            personas.Add(persona);
        }

        reader.Close();
        return personas;
    }
}


### 2.3 Función para Calcular la Desviación Estandar
```csharp
using System;
using System.Collections.Generic;
using System.Linq;

public class Statistics
{
    public decimal CalcularDesviacionEstandar(List<Persona> personas)
    {
        // Obtener los valores de IMC
        List<decimal> imcs = personas.Select(p => p.IMC).ToList();
        
        // Calcular la media del IMC
        decimal media = imcs.Average();

        // Calcular la desviación estándar
        decimal sumatoriaDesviaciones = imcs.Sum(imc => (imc - media) * (imc - media));
        decimal desviacionEstandar = (decimal)Math.Sqrt((double)(sumatoriaDesviaciones / imcs.Count));

        return desviacionEstandar;
    }
}


## 3 Capa de Presentación
```csharp
public class Program
{
    public static void Main(string[] args)
    {
        DatabaseConnection dbConnection = new DatabaseConnection();
        SqlConnection connection = dbConnection.Connect();

        IMCCalculator calculator = new IMCCalculator();
        List<Persona> personas = calculator.ObtenerPersonasConIMC(connection);

        foreach (var persona in personas)
        {
            Console.WriteLine($"Nombre: {persona.Nombre}, Edad: {persona.Edad}, Peso: {persona.Peso}kg, Altura: {persona.Altura}m, IMC: {persona.IMC:F2}");
        }

        connection.Close();
    }
}


### Salida esperada
```sql
Listado de Personas con IMC:
-------------------------------------------
Nombre               Edad  Peso    Altura  IMC    
-------------------------------------------
Juan Pérez           28    72.5    1.75    23.67  
Ana Gómez            34    60.0    1.68    21.26  
Carlos López         45    85.3    1.80    26.32  
Lucía Martínez       22    55.2    1.62    21.03  
-------------------------------------------
-------

Desviación Estándar del IMC: 2.27
